In [1]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=FutureWarning)


import h3
import osmnx as ox
import csv
import json
import pandas as pd
import folium


In [2]:
# https://wiki.openstreetmap.org/wiki/Map_features   tags resources

In [3]:


# Initialize Folium map
m = folium.Map(location=[51.45498615600733, -2.594947745448589], zoom_start=12)

# Define H3 resolution (0 to 15)
resolution = 7

# Define bounding box coordinates
min_lat, min_lon = -2.984188,  51.269558
max_lat, max_lon = -2.416278, 51.56

# Create GeoJSON-like polygon
geo_json_polygon = {
    "type": "Polygon",
    "coordinates": [
        [
            [min_lon, min_lat],
            [min_lon, max_lat],
            [max_lon, max_lat],
            [max_lon, min_lat],
            [min_lon, min_lat]
        ]
    ]
}

# Generate H3 cells covering the map area
hexagons = h3.polyfill(geo_json_polygon, resolution)
data =pd.DataFrame ( hexagons)
data.columns = ['hex7']

In [4]:
def query_osm_for_features(point, tags):
    try:
        # Query OSM for features based on tags within a radius of 500 meters
        # print(point, tags)
        response = ox.geometries.geometries_from_point(point, tags=tags, dist=500)
        return response is not None and len(response) > 0
    except ox._errors.InsufficientResponseError as e:
        #print(f"Error querying features: {e}")
        return False

In [5]:



def add_features_to_dataframe(df,caseNo =2):
    match caseNo:
        case 1:
            df['highway'] = df['hex7'].apply(lambda hex_code: query_osm_for_features(h3.h3_to_geo(hex_code), {'highway': ['motorway', 'trunk', 'primary', 'secondary','residential','motorway_link','trunk_link','primary_link','secondary_link']}))
        case 2:
            df['building'] = df['hex7'].apply(lambda hex_code: query_osm_for_features(h3.h3_to_geo(hex_code), {'building': ['residential' , 'apartments','barracks', 'bungalow', 'detached','house','semidetached_house','stilt_house','terrace']}))
        case 3:
            df['industrial'] = df['hex7'].apply(lambda hex_code: query_osm_for_features(h3.h3_to_geo(hex_code), {'building': ['commercial', 'industrial','kiosk','office','retail','supermarket','warehouse'   ]}))
        case 4:
            df['center_point'] = df['hex7'].apply(lambda hex_code: h3.h3_to_geo(hex_code))

# Example DataFrame with 'hex7' column

#data = pd.read_csv('hexagons_with_attributes.csv')
# Add features to the DataFrame
add_features_to_dataframe(data,1)
add_features_to_dataframe(data,2)
add_features_to_dataframe(data,3)
#add_features_to_dataframe(data,4)
# Print the updated DataFrame
print(data)


C:\Users\Eshkiki\AppData\Local\Temp\ipykernel_6860\3980081970.py:5: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  response = ox.geometries.geometries_from_point(point, tags=tags, dist=500)
C:\Users\Eshkiki\AppData\Local\Temp\ipykernel_6860\3980081970.py:5: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  response = ox.geometries.geometries_from_point(point, tags=tags, dist=500)
C:\Users\Eshkiki\AppData\Local\Temp\ipykernel_6860\3980081970.py:5: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` modu

                hex7  highway  building  industrial
0    871958635ffffff    False     False       False
1    8719580ddffffff     True     False        True
2    8719580daffffff    False     False       False
3    871958655ffffff     True      True        True
4    87195829effffff     True      True       False
..               ...      ...       ...         ...
263  871958651ffffff    False     False       False
264  871958619ffffff     True      True       False
265  871958665ffffff    False      True       False
266  87195805dffffff     True      True       False
267  8719580d9ffffff     True     False        True

[268 rows x 4 columns]


In [6]:
data.to_csv('hexagons_with_attributes.csv')

In [7]:
data.head()

,hex7,highway,building,industrial
0,871958635ffffff,False,False,False
1,8719580ddffffff,True,False,True
2,8719580daffffff,False,False,False
3,871958655ffffff,True,True,True
4,87195829effffff,True,True,False
